In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
path = "/content/drive/MyDrive/Data Science/Project-47 Rain Prediction Using ML models & PyCaret With Deployment (Auto ML)/weatherAUS.csv"

In [ ]:
df = pd.read_csv(path)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# checking null values
df.isna().sum()

In [ ]:
df.nunique()

In [ ]:
df.groupby('MaxTemp').mean()['Evaporation']

In [ ]:
df.groupby('MaxTemp').mean()['Sunshine']

In [ ]:
df['Evaporation'] = df['Evaporation'].fillna(0)
df['Sunshine'] =df['Sunshine'].fillna(0)

In [ ]:
# checking null values
df.isna().sum()

In [ ]:
# selecting datatypes
obj_col = df.select_dtypes('object')
obj_col.head()

In [ ]:
df.select_dtypes('float')

In [ ]:
fill_cloud3pm_1 = df.groupby('Humidity3pm').mean()[['Cloud3pm']]
fill_cloud3pm_2 = df.groupby('Cloud9am').mean()[['Cloud3pm']]


In [ ]:
l = []
for i in range(len(df)):
  if math.isnan(df['Cloud3pm'].iloc[i]):
    if math.isnan(df['Cloud9am'].iloc[i]):
      if math.isnan(df['Humidity3pm'].iloc[i]):
        l.append(df['Cloud3pm'].iloc[i])
      else:
        l.append(fill_cloud3pm_1.loc[df['Humidity3pm'].iloc[i]][0])
    else:
      l.append(fill_cloud3pm_2.loc[df['Cloud9am'].iloc[i]][0])
  else:
    l.append(df['Cloud3pm'].iloc[i])

In [ ]:
df['Cloud3pm_'] = np.array(l)

In [ ]:
fill_cloud9am_1 = df.groupby('Humidity3pm').mean()['Cloud9am']
fill_cloud9am_2 = df.groupby('Cloud3pm').mean()['Cloud9am']

In [ ]:
c2 = []
for i in range(len(df)):
  if math.isnan(df['Cloud9am'].iloc[i]):
    if math.isnan(df['Cloud3pm'].iloc[i]):
      if math.isnan(df['Humidity3pm'].iloc[i]):
        c2.append(df['Cloud9am'].iloc[i])
      else:
        c2.append(fill_cloud9am_1.loc[df['Humidity3pm'].iloc[i]])
    else:
      c2.append(fill_cloud9am_2.loc[df['Cloud3pm'][i]])
  else:
    c2.append(df['Cloud9am'].iloc[i])

In [ ]:
df['Cloud9am_'] = np.array(c2)

In [ ]:
df.isna().sum()*100/len(df)

In [ ]:
df.drop(['Cloud3pm', 'Cloud9am'], axis=1, inplace=True)

In [ ]:
# dropping values

df.dropna(how='any', inplace=True)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
for i in obj_col:
  if i != 'Date':
    print(i)
    print(df[i].value_counts())

    encoder = LabelEncoder().fit(df[i])
    df[i] = encoder.transform(df[i])
    print(dict(zip(encoder.classes_, encoder.transform(encoder.classes_))))

    print('----'*20)

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df['Day'] = df['Date'].apply(lambda x: x.day)
df['Month'] = df['Date'].apply(lambda x: x.month)

In [ ]:
df.drop('Date', axis=1, inplace=True)

## Splitting data 

In [ ]:
# dependent and independent

x = df.drop(['RainToday', 'RainTomorrow'], axis=1)
y = df['RainTomorrow'].values

In [ ]:
scaler = StandardScaler().fit(x)
x = scaler.transform(x)

In [ ]:
# train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=44)

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

## DL Model

In [ ]:
# from tensorflow import keras

In [ ]:
# model = keras.Sequential()

In [ ]:
# model.add(keras.layers.Dense(units=128, activation='relu', input_shape=(22,)))
# model.add(keras.layers.Dropout(0.2))

# model.add(keras.layers.Dense(units=64, activation='relu'))
# model.add(keras.layers.Dropout(0.4))

# model.add(keras.layers.Dense(units=32, activation='relu'))
# model.add(keras.layers.Dropout(0.2))


# model.add(keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
# complie

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# model.fit(x_train, y_train, epochs=5, batch_size=128)

## ML Model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [ ]:
all_models = [LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, XGBClassifier]

In [ ]:
def model_score(model):
  model = model()
  model.fit(x_train, y_train)
  print(type(model).__name__)
  print("Training Score", model.score(x_train, y_train))
  print("Testing Score", model.score(x_test, y_test))
  return model

In [ ]:
models = {}

for i in all_models:
  model = model_score(i)
  models[type(model).__name__] = model
  print('----'*20)

## Auto ML

In [ ]:
# ! pip install pycaret

In [ ]:
from pycaret.classification import *

In [ ]:
exp_name = setup(data = df,  target = 'RainTomorrow')

In [ ]:
best_model = compare_models()

In [ ]:
print(best_model)

In [ ]:
evaluate_model(best_model)

In [ ]:
plot_model(best_model, plot = 'confusion_matrix')

In [ ]:
model = create_model(best_model)

In [ ]:
save_model(best_model, 'my_best_pipeline')